In [1]:
import re
import time
import pickle
import numpy as np

from edit_distance import SequenceMatcher
import torch
from dataset import SpeechDataset
import matplotlib.pyplot as plt

from neural_decoder.dataset import getDatasetLoaders
import neural_decoder.lm_utils as lmDecoderUtils
from neural_decoder.model import GRUDecoder
import pickle
import argparse
import matplotlib.pyplot as plt
from neural_decoder.dataset import getDatasetLoaders
import neural_decoder.lm_utils as lmDecoderUtils
from neural_decoder.lm_utils import build_llama_1B
from neural_decoder.model import GRUDecoder
from neural_decoder.bit import BiT_Phoneme
import pickle
import argparse
from lm_utils import _cer_and_wer
import json
import os
import copy

/home3/ebrahim2/miniconda3/envs/speech-bci/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def convert_sentence(s):
    s = s.lower()
    charMarks = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
                 "'", ' ']
    ans = []
    for i in s:
        if(i in charMarks):
            ans.append(i)
    
    return ''.join(ans)

In [3]:
base_dir = "/home3/skaasyap/willett"

load_lm = True
# LM decoding hyperparameters
acoustic_scale = 0.8
blank_penalty = np.log(2)

run_for_llm = False

if run_for_llm:
    return_n_best = True
    rescore = False
    nbest = 100
    print("RUNNING IN LLM MODE")
else:
    return_n_best = False
    rescore = False
    nbest = 1
    print("RUNNING IN N-GRAM MODE")

RUNNING IN N-GRAM MODE


In [4]:
if load_lm: 
        
    lmDir = base_dir +'/lm/languageModel'
    ngramDecoder = lmDecoderUtils.build_lm_decoder(
        lmDir,
        acoustic_scale=acoustic_scale, #1.2
        nbest=nbest,
        beam=18
    )
    print("loaded LM")
    

loaded LM


I0514 18:57:10.989542 1595699 brain_speech_decoder.h:52] Reading fst /home3/skaasyap/willett/lm/languageModel/TLG.fst
I0514 18:59:40.748227 1595699 brain_speech_decoder.h:81] Reading symbol table /home3/skaasyap/willett/lm/languageModel/words.txt


# Get model predictions
## Always check to make sure val perf matches wandb

In [63]:
output_file = 'leia'
    
device = "cuda:2"

if output_file == 'obi':
    model_storage_path = '/data/willett_data/outputs/'
elif output_file == 'leia':
    model_storage_path = '/data/willett_data/leia_outputs/'
    
models_to_run = ['neurips_transformer_time_masked_held_out_days_2', 
                 'neurips_transformer_time_masked_held_out_days_1',
                 'neurips_transformer_time_masked_held_out_days']

shared_output_file = 'corrected_memo_transformer'

if len(shared_output_file) > 0:
    print("Writing to shared output file")
    write_mode = "a"
else:
    write_mode = "w"
    
seeds_list = [0, 1, 2, 3]
partition = "competition" # "test"
run_lm = True

comp_on_reduced = True
fill_max_day = False
memo = True
memo_epochs = 1
memo_augs = 16
memo_lr = [3e-5, 6e-5, 6e-5]
skip_days = [[], [], []]

day_edit_distance = 0
day_seq_length = 0
prev_day = None

memo_seeds = np.arange(1)

if partition == 'test':
    saveFolder_transcripts = "/data/willett_data/model_transcriptions/"
else:
    saveFolder_transcripts = "/data/willett_data/model_transcriptions_comp/"
    

day_cer_dict = {}
total_wer_dict = {}

for seed in seeds_list:
    
    day_cer_dict[seed] = []
    total_wer_dict[seed] = []
            
    for mn, model_name_str in enumerate(models_to_run):
        
        modelPath = f"{model_storage_path}{model_name_str}_seed_{seed}"
        
        if len(shared_output_file) > 0:
            output_file = f"{shared_output_file}_seed_{seed}"
            print(output_file)
        else:
            output_file = f"{model_name_str}_seed_{seed}"
            
        print(f"Running model: {model_name_str}_seed_{seed}")
            
        with open(modelPath + "/args", "rb") as handle:
            args = pickle.load(handle)
            
        if args['datasetPath'].rsplit('/', 1)[-1] == 'data_log_both':
            data_file = '/data/willett_data/ptDecoder_ctc_both'
            
        elif args['datasetPath'].rsplit('/', 1)[-1] == 'data':
            data_file = '/data/willett_data/ptDecoder_ctc'
            
        elif args['datasetPath'].rsplit('/', 1)[-1] == 'data_log_both_held_out_days':
            data_file = '/data/willett_data/ptDecoder_ctc_both_held_out_days'
            
        elif args['datasetPath'].rsplit('/', 1)[-1] == 'data_log_both_held_out_days_1':
            data_file = '/data/willett_data/ptDecoder_ctc_both_held_out_days_1'
            
        elif args['datasetPath'].rsplit('/', 1)[-1] == 'data_log_both_held_out_days_2':
            data_file = '/data/willett_data/ptDecoder_ctc_both_held_out_days_2'
            
        else:
            data_file = args['datasetPath']
            
        trainLoaders, testLoaders, loadedData = getDatasetLoaders(
            data_file, 8
        )
        
        # if true, model is a GRU
        if 'nInputFeatures' in args.keys():
            
            if 'max_mask_pct' not in args:
                args['max_mask_pct'] = 0
            if 'num_masks' not in args:
                args['num_masks'] = 0
            if 'input_dropout' not in args:
                args['input_dropout'] = 0
                
            print("Loading GRU")
            model = GRUDecoder(
                neural_dim=args["nInputFeatures"],
                n_classes=args["nClasses"],
                hidden_dim=args["nUnits"],
                layer_dim=args["nLayers"],
                nDays=args['nDays'],
                dropout=args["dropout"],
                device=device,
                strideLen=args["strideLen"],
                kernelLen=args["kernelLen"],
                gaussianSmoothWidth=args["gaussianSmoothWidth"],
                bidirectional=args["bidirectional"],
                input_dropout=args['input_dropout'], 
                max_mask_pct=args['max_mask_pct'],
                num_masks=args['num_masks']
            ).to(device)

        else:
            
            if 'mask_token_zero' not in args:
                args['mask_token_zero'] = False
                            
            # Instantiate model
            # set training relevant parameters for MEMO, doesn't matter for other runs because they are 
            # only run in eval mode.
            model = BiT_Phoneme(
                patch_size=args['patch_size'],
                dim=args['dim'],
                dim_head=args['dim_head'],
                nClasses=args['nClasses'],
                depth=args['depth'],
                heads=args['heads'],
                mlp_dim_ratio=args['mlp_dim_ratio'],
                dropout=0,
                input_dropout=0,
                look_ahead=args['look_ahead'],
                gaussianSmoothWidth=args['gaussianSmoothWidth'],
                T5_style_pos=args['T5_style_pos'],
                max_mask_pct=0.05,
                num_masks=20, 
                mask_token_zeros=args['mask_token_zero'], 
                num_masks_channels=0, 
                max_mask_channels=0, 
                dist_dict_path=0, 
            ).to(device)
            
            
        ckpt_path = modelPath + '/modelWeights'
        model.load_state_dict(torch.load(ckpt_path, map_location=device), strict=True)
        model = model.to(device)
        
        if memo: 
            
            optimizer = torch.optim.AdamW(model.parameters(), lr=memo_lr[mn], weight_decay=0, 
                                    betas=(args['beta1'], args['beta2']))
            
            for name, p in model.named_parameters():
                
                if name in {
                    "to_patch_embedding.1.weight",
                    "to_patch_embedding.1.bias",
                    "to_patch_embedding.2.weight",
                    "to_patch_embedding.2.bias",
                    "to_patch_embedding.3.weight",
                    "to_patch_embedding.3.bias"
                }:
                    p.requires_grad = True
                else:
                    p.requires_grad = False
        
        model.eval()

        model_outputs = {
            "logits": [],
            "logitLengths": [],
            "trueSeqs": [],
            "transcriptions": [],
        }
        
        total_edit_distance = 0
        total_seq_length = 0

        if partition == "competition":
            
            if comp_on_reduced:
                testDayIdxs = np.arange(5)
            else:
                testDayIdxs = [4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19, 20]
                
        elif partition == "test":
            
            testDayIdxs = range(len(loadedData[partition])) 
            
        ground_truth_sentences = []
        
        #print("RESTRICTED DAYS: ", args['restricted_days'])
        
        for i, testDayIdx in enumerate(testDayIdxs):
            
            if len(skip_days[mn]) > 0:
                if testDayIdx in skip_days[mn]:
                    print("SKIPPING DAY: ", testDayIdx)
                    continue
            
            if len(args['restricted_days']) > 0:
                if testDayIdx not in args['restricted_days']:
                    continue
                
            test_ds = SpeechDataset([loadedData[partition][i]])
            test_loader = torch.utils.data.DataLoader(
                test_ds, batch_size=1, shuffle=False, num_workers=0
            )
            
            for j, (X, y, X_len, y_len, _) in enumerate(test_loader):
                        
                X, y, X_len, y_len, dayIdx = (
                    X.to(device),
                    y.to(device),
                    X_len.to(device),
                    y_len.to(device),
                    torch.tensor([testDayIdx], dtype=torch.int64).to(device),
                )
                
                if fill_max_day:
                    dayIdx.fill_(args['maxDay'])
                    
                if memo: 
                                        
                    model.train()
                    
                    for _ in range(memo_epochs):
                        
                        logits_aug = model.forward(X, X_len, testDayIdx, memo_augs)  # [memo_augs, T, D]
                        probs_aug = torch.nn.functional.softmax(logits_aug, dim=-1)          # [memo_augs, T, D]
                        marginal_probs = probs_aug.mean(dim=0)                               # [T, D]

                        adjustedLens = model.compute_length(X_len)
                        marginal_probs = marginal_probs[:adjustedLens]

                        loss = - (marginal_probs * marginal_probs.log()).sum(dim=-1).mean()

                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                        
                    model.eval()
                    
                with torch.no_grad():
                    
                    pred = model.forward(X, X_len, dayIdx)
                
                if hasattr(model, 'compute_length'):
                    adjustedLens = model.compute_length(X_len)
                else:
                    adjustedLens = ((X_len - model.kernelLen) / model.strideLen).to(torch.int32)
                    
                for iterIdx in range(pred.shape[0]):
                    
                    trueSeq = np.array(y[iterIdx][0 : y_len[iterIdx]].cpu().detach())
                    model_outputs["logits"].append(pred[iterIdx].cpu().detach().numpy())
                    
                    model_outputs["logitLengths"].append(
                        adjustedLens[iterIdx].cpu().detach().item()
                    )
                    
                    model_outputs["trueSeqs"].append(trueSeq)
                    
                    decodedSeq = torch.argmax(
                        torch.tensor(pred[iterIdx, 0 : adjustedLens[iterIdx], :]),
                        dim=-1,
                    ) 
                    
                    decodedSeq = torch.unique_consecutive(decodedSeq, dim=-1)
                    decodedSeq = decodedSeq.cpu().detach().numpy()
                    decodedSeq = np.array([i for i in decodedSeq if i != 0])
                    
                    matcher = SequenceMatcher(
                        a=trueSeq.tolist(), b=decodedSeq.tolist()
                    )
                    
                    total_edit_distance += matcher.distance()
                    total_seq_length += len(trueSeq)
                    
                    day_edit_distance += matcher.distance()
                    day_seq_length += len(trueSeq)
                    
                transcript = loadedData[partition][i]["transcriptions"][j].strip()
                transcript = re.sub(r"[^a-zA-Z\- \']", "", transcript)
                transcript = transcript.replace("--", "").lower()
                model_outputs["transcriptions"].append(transcript)
                
            cer_day = day_edit_distance / day_seq_length
            day_cer_dict[seed].append(cer_day)
            # print("CER DAY: ", cer_day)
            day_edit_distance = 0 
            day_seq_length = 0

        cer = total_edit_distance / total_seq_length
        
        print("Model performance: ", cer)

        if run_lm:
            
            print("Running n-gram LM")
            
            llm_outputs = []
            start_t = time.time()
            nbest_outputs = []
            
            for j in range(len(model_outputs["logits"])):
                
                logits = model_outputs["logits"][j]
                
                logits = np.concatenate(
                    [logits[:, 1:], logits[:, 0:1]], axis=-1
                )  # Blank is last token
                
                logits = lmDecoderUtils.rearrange_speech_logits(logits[None, :, :], has_sil=True)
                
                nbest = lmDecoderUtils.lm_decode(
                    ngramDecoder,
                    logits[0],
                    blankPenalty=blank_penalty,
                    returnNBest=return_n_best,
                    rescore=rescore,
                )
                
                nbest_outputs.append(nbest)
                
            time_per_sample = (time.time() - start_t) / len(model_outputs["logits"])
            print(f"N-gram decoding took {time_per_sample} seconds per sample")
            
            if run_for_llm:
                print("SAVING OUTPUTS FOR LLM")
                with open(f"{saveFolder_transcripts}{model_name_str}_seed_{seed}_model_outputs.pkl", "wb") as f:
                    pickle.dump(model_outputs, f)
                    
                with open(f"{saveFolder_transcripts}{model_name_str}_seed_{seed}_nbest.pkl", "wb") as f:
                    pickle.dump(nbest_outputs, f)
                
            else:
                # just get perf with greedy decoding
                for i in range(len(model_outputs["transcriptions"])):
                    model_outputs["transcriptions"][i] = model_outputs["transcriptions"][i].strip()
                    nbest_outputs[i] = nbest_outputs[i].strip()
                
                # lower case + remove puncs
                for i in range(len(model_outputs["transcriptions"])):
                    model_outputs["transcriptions"][i] = convert_sentence(model_outputs["transcriptions"][i])

                cer, wer = _cer_and_wer(nbest_outputs, model_outputs["transcriptions"], 
                                    outputType='speech', returnCI=True)

                print("CER and WER after 3-gram LM: ", cer, wer)       
                
                out_file = os.path.join(saveFolder_transcripts, output_file)   # no extension per your spec
                
                with open(out_file + '.txt', write_mode, encoding="utf-8") as f:
                    f.write("\n".join(nbest_outputs)+ "\n")   # one line per LLM output  
                    
                total_wer_dict[seed] = wer
                
if memo:
    with open(f"/data/willett_data/paper_results/{model_name_str}_per_memo.pkl", "wb") as f:
        pickle.dump(day_cer_dict, f)
        
    with open(f"/data/willett_data/paper_results/{model_name_str}_wer_memo.pkl", "wb") as f:
        pickle.dump(total_wer_dict, f)
        
else:
    
    with open(f"/data/willett_data/paper_results/{model_name_str}_per.pkl", "wb") as f:
        pickle.dump(day_cer_dict, f)
        
    with open(f"/data/willett_data/paper_results/{model_name_str}_wer.pkl", "wb") as f:
        pickle.dump(total_wer_dict, f)

Writing to shared output file
corrected_memo_transformer_seed_0
Running model: neurips_transformer_time_masked_held_out_days_2_seed_0


/tmp/ipykernel_1595699/2714812422.py:270: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(pred[iterIdx, 0 : adjustedLens[iterIdx], :]),


Model performance:  2.445
Running n-gram LM
N-gram decoding took 0.033970991969108584 seconds per sample
CER and WER after 3-gram LM:  (2.3953125, 2.2962421875000003, 2.4946875) (2.65625, 2.5575, 2.755)
corrected_memo_transformer_seed_0
Running model: neurips_transformer_time_masked_held_out_days_1_seed_0
Model performance:  2.84875
Running n-gram LM
N-gram decoding took 0.01881837785243988 seconds per sample
CER and WER after 3-gram LM:  (3.07875, 2.968125, 3.1909375) (3.11625, 3.01375, 3.22125)
corrected_memo_transformer_seed_0
Running model: neurips_transformer_time_masked_held_out_days_seed_0
Model performance:  2.9646875
Running n-gram LM
N-gram decoding took 0.019480100870132445 seconds per sample
CER and WER after 3-gram LM:  (3.2103125, 3.1068671875000002, 3.3128125) (3.26625, 3.16625, 3.36625)
corrected_memo_transformer_seed_1
Running model: neurips_transformer_time_masked_held_out_days_2_seed_1
Model performance:  2.4678125
Running n-gram LM
N-gram decoding took 0.03165530323

In [60]:
print(total_wer_dict)

{0: (0.2843184754521964, 0.27441247863650914, 0.29429395482713483), 1: (0.28625645994832044, 0.2764180798693545, 0.2963570126909906), 2: (0.28141149870801035, 0.2713125020955371, 0.2914014531184867), 3: (0.2777777777777778, 0.26769863403614913, 0.28783718852237616)}


In [61]:
print(day_cer_dict)

{0: [0.19296577946768062, 0.2064954817929677, 0.20115387927322828, 0.2408321950194389, 0.2471046650070354], 1: [0.20428620808848946, 0.20891115683993916, 0.20864548350985965, 0.24062204476200483, 0.24396579716419525], 2: [0.1858797096439682, 0.19906951775968507, 0.20390941186601222, 0.23085005779132078, 0.24818703322870442], 3: [0.18363290701693744, 0.19862217052876444, 0.19770946353224833, 0.23200588420720816, 0.2390951401666847]}
